In [3]:
!pip install transformers
!pip install torchmetrics
!pip install pytorch_lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 32.2 MB/s 
     |████████████████████████████████| 596 kB 67.8 MB/s 
     |████████████████████████████████| 101 kB 10.6 MB/s 
     |████████████████████████████████| 6.6 MB 62.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 419 kB 34.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 585 kB 33.9 MB/s 
     |████████████████████████████████| 140 kB 72.5 MB/s 
     |████████████████████████████████| 1.1 MB 54.6 MB/s 
     |████████████████████████████████| 144 kB 75.2 MB/s 
     |██████████████████████████

In [4]:
from google.colab import drive
from torch import nn
from transformers import BertTokenizer, BertModel
import torch
import csv
from torch.utils.data import Dataset
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AdamW
from torchmetrics import Accuracy
import os
from torch.utils.data.dataloader import DataLoader
from torch.nn.utils.rnn import pad_sequence
import tqdm
import time
import math
import numpy as np
from transformers import DataCollatorForLanguageModeling
import pytorch_lightning as pl

In [5]:
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/data'

train_data_path = f'{data_path}/train'
true_train_path = f'{train_data_path}/true.csv'
false_train_path = f'{train_data_path}/false.csv'

Mounted at /content/drive


In [6]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

true_label = 1
false_label = 0

true_saved_model_path = '/content/drive/MyDrive/models/true_bert_lm.pt'
false_saved_model_path = '/content/drive/MyDrive/models/false_bert_lm.pt'

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
class LMSpellCheckingDataset(Dataset):

    def __init__(self, tokenizer, data_paths, labels, batch_size=32):
        self.dataset = []
        for i in range(len(data_paths)):
          data_path = data_paths[i]
          with open(data_path, 'r', encoding='utf-8') as file:
              data = csv.reader(file)
              for item in data:
                self.dataset.append((item[0], labels[i]))
        self.tokenizer = tokenizer
        self.batch_size = batch_size

    def __getitem__(self, idx):
        if (self.tokenizer == None):
          raise Exception('Tokenizer cannot be null')

        tweet, label = self.dataset[idx]
        tokenized_tweet = self.tokenizer(tweet)
        input_ids = tokenized_tweet['input_ids']
        attention_mask = tokenized_tweet['attention_mask']

        return torch.tensor(input_ids)

    def __len__(self):
      return len(self.dataset)

class BertLM(pl.LightningModule):

    def __init__(self, class_name):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')
        self.epoch_number = 0
        self.class_name = class_name

    def forward(self, input_ids, labels):
        return self.bert(input_ids=input_ids,labels=labels)

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        labels = batch["labels"]
        outputs = self(input_ids=input_ids, labels=labels)
        loss = outputs[0]
        return {"loss": loss}

    def training_epoch_end(self, outputs):
        super().training_epoch_end(outputs)
        mean_loss = 0
        n_batch  = len(outputs)
        for i in range(n_batch):
            mean_loss += outputs[i]['loss'].cpu().numpy() / n_batch
        print(f"End of epoch {self.epoch_number} with mean loss '{mean_loss}' on label {self.class_name}.", "fine_tuning")
        self.epoch_number += 1
    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=1e-5)

class BertLMPred(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = BertForMaskedLM.from_pretrained('bert-base-uncased')

    def forward(self, input_ids, labels=None):
        return self.bert(input_ids=input_ids,labels=labels)

In [ ]:
MASK = '[MASK]'
SEP = '[SEP]'
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_batch(batch, tokenizer):
    return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]

def get_init_text(seed_sentence, max_len, tokenizer, batch_size=1):
    batch = [seed_sentence + [MASK] * max_len + [SEP] for _ in range(batch_size)]
    return tokenize_batch(batch, tokenizer)

def untokenize_batch(batch, tokenizer):
    return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]

def generate_step(out, gen_idx, temperature=None, top_k=0, 
                  sample=False, return_list=True):
    """ Generate a word from from out[gen_idx]
    
    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k 
    """
    logits = out.logits[:, gen_idx]
    if temperature is not None:
        logits = logits / temperature
    if top_k > 0:
        kth_vals, kth_idx = logits.topk(top_k, dim=-1)
        dist = torch.distributions.categorical.Categorical(logits=kth_vals)
        idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
    elif sample:
        dist = torch.distributions.categorical.Categorical(logits=logits)
        idx = dist.sample().squeeze(-1)
    else:
        idx = torch.argmax(logits, dim=-1)
    return idx.tolist() if return_list else idx


def parallel_sequential_generation(seed_text, tokenizer, model, batch_size=1, 
                                   max_len=15, top_k=0, temperature=None, 
                                   max_iter=300, burnin=200, cuda=False, 
                                   print_every=10, verbose=True):
    """ Generate for one random position at a timestep
    
    args:
        - burnin: during burn-in period, sample from full distribution; afterwards take argmax
    """
    seed_len = len(seed_text)
    batch = get_init_text(seed_text, max_len, tokenizer, batch_size=batch_size)
    mask_id = tokenizer.convert_tokens_to_ids([MASK])[0]
    for ii in range(max_iter):
        kk = np.random.randint(0, max_len)
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = mask_id
        inp = torch.tensor(batch).cuda() if cuda else torch.tensor(batch)
        out = model(inp)
        topk = top_k if (ii >= burnin) else 0
        idxs = generate_step(out, gen_idx=seed_len+kk, top_k=topk, temperature=temperature, sample=(ii < burnin))
        #if idxs is a single number
        if isinstance(idxs, int):
            idxs = [idxs]
        for jj in range(batch_size):
            batch[jj][seed_len+kk] = idxs[jj]
            
    return untokenize_batch(batch, tokenizer)


def generate(tokenizer, n_samples, class_name, model, seed_text="[CLS]", 
             batch_size=1, max_len=25, sample=True, top_k=100, temperature=1.0, 
             burnin=200, max_iter=500, cuda=False, print_every=1):
    # main generation function to call
    sentences = []
    n_batches = math.ceil(n_samples / batch_size)
    start_time = time.time()
    for batch_n in range(n_batches):
        batch = parallel_sequential_generation(seed_text, tokenizer, model, 
                                               max_len=max_len, top_k=top_k,
                                               temperature=temperature, 
                                               burnin=burnin, max_iter=max_iter, 
                                               cuda=cuda, verbose=False)
        
        if (batch_n + 1) % print_every == 0:
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time))
            print("Finished batch %d in %.3fs" % (batch_n + 1, time.time() - start_time), "fine_tuning")
            start_time = time.time()
        
        sentences += batch
    return sentences

def standardize_sentence(sent):
    sentence = []
    current_word = sent[0]
    for i in range(1, len(sent)):
        token = sent[i]
        if(token[0:2] == '##'):
            current_word += token[2:]
        else:
            sentence.append(current_word)
            current_word = token
    sentence.append(current_word)
    return sentence

def fine_tune_LM(class_name, tokenizer, epochs, batch_size, 
                 train_paths, labels, save_url=None, mlm_prob=0.25, 
                 use_gpu=True):
  
    dataset = LMSpellCheckingDataset(tokenizer, train_paths, labels)
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, 
                                                    mlm=True, 
                                                    mlm_probability=mlm_prob)
    train_loader = DataLoader(dataset, 
                              batch_size=batch_size, 
                              collate_fn=data_collator)
    model = BertLM(class_name)
    #using CPU
    if use_gpu:
        trainer = pl.Trainer(max_epochs=epochs, 
                             checkpoint_callback=False, 
                             logger=False, 
                             gpus=1)
    else:
        trainer = pl.Trainer(max_epochs=epochs, 
                             checkpoint_callback=False, 
                             logger=False)
        
    print(f"Start fine tuning BERT masked LM on class {class_name}", "fine_tuning")
    trainer.fit(model, train_loader)
    if save_url is not None:
        print(f"Finished training. Saving model in {save_url}", "fine_tuning")
        torch.save(model.state_dict(), save_url)

def fine_tune(tokenizer):
    labels = ['false', 'true']
    bert_model_url = "/content/drive/MyDrive/models/"
    reports_dir = "/content/drive/MyDrive/reports/"
    if not os.path.exists(reports_dir):
        os.makedirs(reports_dir, exist_ok=True)
    labels = [item for item in labels]
    
    for label in labels:
        print('----------\n')
        print(f"Generating sentences for {label} label ...", "fine_tuning")
        model_url = os.path.join(bert_model_url, f"{label}_bert_lm.pt")
        model = BertLMPred()
        model.load_state_dict(torch.load(model_url))
        model = model.cuda()
        sentences = generate(tokenizer, 10, label, model, max_len=10, seed_text='[CLS]'.split(), cuda=True)
        out_url = reports_dir + f"{label}.txt"
        with open(out_url, 'w') as outf:
            for sent in sentences:
                sent = standardize_sentence(sent)
                outf.write(' '.join(sent).replace('[', '<').replace(']', '>'))
                outf.write("\\\\")
                outf.write("\n")
        print(f"Sentences for {label} saved to {out_url}.", "fine_tuning")
        print(sentences)
        print('----------\n')
        
fine_tune_LM('true', tokenizer, 20, 32, [true_train_path], [true_label], 
             save_url=true_saved_model_path)

fine_tune_LM('false', tokenizer, 20, 32, [false_train_path], [false_label], 
             save_url=false_saved_model_path)

fine_tune(tokenizer)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=False)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=False)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated

Start fine tuning BERT masked LM on class true fine_tuning


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name | Type            | Params
-----------------------------------------
0 | bert | BertForMaskedLM | 109 M 
-----------------------------------------
109 M     Trainable params
0         Non-trainable params
109 M     Total params
438.057   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


Finished training. Saving model in /content/drive/MyDrive/models/true_bert_lm.pt fine_tuning
